In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S

from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(1)

In [3]:
print(os.listdir("../test_img"))

['depression', 'violence', 'happiness']


In [5]:
images_dir = "../test_img"
emotion_dirs = os.listdir(images_dir)
emotion_dirs

['depression', 'violence', 'happiness']

In [6]:
86+69 + 36

191

In [7]:
weight = {'depression' : 86, 'violence': 69, 'happiness':36}
class_weights = {}
for i in range(3):
    w = 191 / (3 * weight[emotion_dirs[i]])
    class_weights[i] = w 
class_weights

{0: 0.7403100775193798, 1: 0.9227053140096618, 2: 1.7685185185185186}

In [14]:
batch_size = 16
train_input_shape = (224, 224, 3)
n_classes = 3

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=emotion_dirs
                                                   )

valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=emotion_dirs
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

Found 154 images belonging to 3 classes.
Found 38 images belonging to 3 classes.
Total number of batches = 9 and 2


In [15]:
# Load pre-trained model
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=train_input_shape)

for layer in base_model.layers:
    layer.trainable = True

In [16]:
# Add layers at the end
X = base_model.output
X = Flatten()(X)

X = Dense(512, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(16, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

output = Dense(n_classes, activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=output)

In [17]:
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

In [18]:
n_epoch = 20

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              verbose=1, mode='auto')

In [19]:
# Train the model - all layers
history1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

/tmp/ipykernel_3618682/4005977698.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,


Epoch 1/20
9/9 [==============================] - 180s 6s/step - loss: 1.4483 - accuracy: 0.2464 - val_loss: 3.8418 - val_accuracy: 0.4062 - lr: 1.0000e-04
Epoch 2/20
9/9 [==============================] - 157s 5s/step - loss: 1.0977 - accuracy: 0.3913 - val_loss: 3.0036 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 3/20
9/9 [==============================] - 171s 6s/step - loss: 0.9388 - accuracy: 0.4444 - val_loss: 1.8894 - val_accuracy: 0.2188 - lr: 1.0000e-04
Epoch 4/20
9/9 [==============================] - 175s 6s/step - loss: 0.8645 - accuracy: 0.5435 - val_loss: 1.6261 - val_accuracy: 0.2188 - lr: 1.0000e-04
Epoch 5/20
9/9 [==============================] - 176s 6s/step - loss: 0.6902 - accuracy: 0.6667 - val_loss: 1.4566 - val_accuracy: 0.3125 - lr: 1.0000e-04
Epoch 6/20
9/9 [==============================] - 179s 6s/step - loss: 0.5865 - accuracy: 0.7029 - val_loss: 1.1501 - val_accuracy: 0.4688 - lr: 1.0000e-04
Epoch 7/20
9/9 [==============================] - 189s 7s/step -

In [20]:
# Freeze core ResNet layers and train again 
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[:50]:
    layer.trainable = True

optimizer = Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

n_epoch = 50
history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr, early_stop],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

/home/jupyter-k7a506/.local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/tmp/ipykernel_3618682/2442743920.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,


Epoch 1/50
9/9 [==============================] - 249s 19s/step - loss: 0.2472 - accuracy: 0.9638 - val_loss: 1.3144 - val_accuracy: 0.5312 - lr: 1.0000e-04
Epoch 2/50
9/9 [==============================] - 248s 18s/step - loss: 0.2233 - accuracy: 0.9928 - val_loss: 1.1830 - val_accuracy: 0.5312 - lr: 1.0000e-04
Epoch 3/50
9/9 [==============================] - 258s 22s/step - loss: 0.2370 - accuracy: 0.9855 - val_loss: 1.0395 - val_accuracy: 0.5625 - lr: 1.0000e-04
Epoch 4/50
9/9 [==============================] - 250s 21s/step - loss: 0.2605 - accuracy: 0.9638 - val_loss: 1.2882 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 5/50
9/9 [==============================] - 253s 18s/step - loss: 0.2422 - accuracy: 0.9653 - val_loss: 1.2327 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 6/50
9/9 [==============================] - 257s 19s/step - loss: 0.2444 - accuracy: 0.9783 - val_loss: 1.3022 - val_accuracy: 0.4375 - lr: 1.0000e-04
Epoch 7/50
9/9 [==============================] - 272s 23s

In [21]:
model.save('efficientnet_v2_s_childdata.h5')